## Intro
This notebook aggregates the environmental data by event whereas before we were looking at the data by date. 

### Calculate number of locations that flooded

In [1]:
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from focus_intersection import subset_floods, flood_df, subset_locations
from main_db_script import data_dir, db_filename
from hr_db_scripts.main_db_script import get_table_for_variable_code, get_db_table_as_df
import pandas as pd
import numpy as np
import re
import sqlite3
import math
con = sqlite3.connect(db_filename)
pd.options.mode.chained_assignment = None  # default='warn'

In this case we are just focusing on the subset of points that is in the downtown area thus the "subset_floods."

In [2]:
flood_events = pd.read_csv('flood_events.csv')
flood_events['event_date'] = pd.to_datetime(flood_events['event_date'])
flood_events['event_name'] = flood_events['event_name'].str.strip()
flood_events['dates'] = pd.to_datetime(flood_events['dates'])

In [3]:
grouped = flood_events.groupby(['event_date', 'event_name'])

Get the number of dates the event spanned, the number of unique locations that were flooded during the event and the total number of locations flooded on all event dates. 

In [4]:
event_total_flooded = grouped.size()
event_dates = grouped['dates'].unique()
num_event_dates = grouped['dates'].nunique()
num_locations = grouped['location'].nunique()

In [5]:
event_df = pd.concat([event_dates, event_total_flooded, num_event_dates, num_locations], axis=1)
event_df.columns = ['dates', 'num_flooded', 'num_dates', 'num_locations']

In [6]:
event_df.tail()

,,dates,num_flooded,num_dates,num_locations
event_date,event_name,,,,
2016-07-31,Thunderstorm-2016-07-31,[2016-07-31T00:00:00.000000000],8,1,8
2016-09-02,HERMINE-2016-09-02,"[2016-09-03T00:00:00.000000000, 2016-09-04T00:...",40,2,40
2016-09-19,HEAVY RAIN-2016-09-19,[2016-09-19T00:00:00.000000000],11,1,11
2016-09-20,unnamed-2016-09-20,"[2016-09-21T00:00:00.000000000, 2016-09-22T00:...",101,4,101
2016-10-05,Hurricane Matthew-2016-10-05,"[2016-10-05T00:00:00.000000000, 2016-10-08T00:...",111,7,109


### Where num_flooded does not equal num_locations _investigation_
Let's checkout one of the events where the num_flooded is greater than the num_locations. I would expect this to mean that one location was flooded on multiple days of the same event. But for '2014-07-24' the event is only on one day so that isn't what I expected.

In [7]:
idx = pd.IndexSlice
event_df.sort_index(inplace=True)
event_df.loc[idx['2014-07-24', :], :]

,,dates,num_flooded,num_dates,num_locations
event_date,event_name,,,,
2014-07-24,unnamed-2014-07-24,"[2014-07-24T00:00:00.000000000, 2014-07-26T00:...",18,2,17


In [8]:
fl_724 = flood_events[flood_events['dates'] == '2014-07-24']
fl_724[fl_724['location'].duplicated(keep=False)]

,Unnamed: 0,location,eventType,dt,event_name,event_date,dates
444,444,HAMPTON BOULEVARD & W 21ST STREET,Flooded street,2014-07-24 20:29:25,unnamed-2014-07-24,2014-07-24,2014-07-24
445,445,HAMPTON BOULEVARD & W 21ST STREET,Flooded underpass,2014-07-24 20:29:25,unnamed-2014-07-24,2014-07-24,2014-07-24


So _here's_ what is happening. The location name is the same in two rows but there are two different event types: "flooded street" and "flooded underpass."
Now that I think about it, that may explain all the differences between the num_location and num_flooded columns. Let's try another one, this time one that spans more than one day: Irene.

In [9]:
event_df.sort_index(inplace=True)
event_df.loc[idx[:, 'Irene-2011-08-27'], :]

,,dates,num_flooded,num_dates,num_locations
event_date,event_name,,,,
2011-08-27,Irene-2011-08-27,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110,4,108


In [10]:
irene = flood_events[flood_events['event_name'].str.contains('Irene')].sort_values('location')
irene[irene['location'].duplicated(keep=False)]

,Unnamed: 0,location,eventType,dt,event_name,event_date,dates
181,181,1000 BLOCK OF E VIRGINIA BEACH BOULEVARD,Flooded street,2011-08-28 04:18:01,Irene-2011-08-27,2011-08-27,2011-08-28
182,182,1000 BLOCK OF E VIRGINIA BEACH BOULEVARD,Flooded underpass,2011-08-28 04:18:01,Irene-2011-08-27,2011-08-27,2011-08-28
185,185,E 21ST STREET & MONTICELLO AVENUE,Flooded underpass,2011-08-28 08:44:35,Irene-2011-08-27,2011-08-27,2011-08-28
104,104,E 21ST STREET & MONTICELLO AVENUE,Flooded street,2011-08-27 06:08:00,Irene-2011-08-27,2011-08-27,2011-08-27


Looks like that's it. Which is not what I was hoping to show. I was thinking that that tell me something about the variety of locations that were flooded over the days but that's not the case.

Let's try this one more time with Hurricane Joaquin

In [11]:
jqn = flood_df[flood_df['event'].str.contains('Joaquin')]

In [12]:
jqn[jqn['location'].duplicated(keep=False)]

,Unnamed: 0,﻿recid,location,event,eventType,xcoord,ycoord,dt,_date,_time


So that is interesting. Even though for hurricanes Matthew and Joaquin, the seven and six days respectively, none
of the flooded locations were reported twice for one event. Very interesting. So to me, this means we really should be looking at these things by 'event' and not by '\_date'. It also means that the num_locations col doesn't add any information. So imma delete that.

In [13]:
del event_df['num_locations']

### Looking into date in "event" column versus dates in "\_date" column
Sometimes the date listed in the "event" column is quite different than the date(s) listed in the "\_date" column. A good example of this is the event "unnamed (2/25/2016)" where the dates in the "\_date" column are 2016-05-05, 2016-05-06, and 2016-05-31"

In [14]:
flood_df[flood_df['event'].str.contains('2/25/2016')]

,Unnamed: 0,﻿recid,location,event,eventType,xcoord,ycoord,dt,_date,_time
760,760,4815,19TH BAY STREET & PLEASANT AVENUE,unnamed (2/25/2016),Flooded street,1.215646e+07,3.505946e+06,2016-05-05 20:43:53.000,2016-05-05,2016-05-05 20:43:53.000
761,761,4816,20TH BAY STREET & PLEASANT AVENUE,unnamed (2/25/2016),Flooded street,1.215680e+07,3.505887e+06,2016-05-05 20:44:41.000,2016-05-05,2016-05-05 20:44:41.000
762,762,4817,BOUSH STREET & W OLNEY ROAD,unnamed (2/25/2016),Flooded street,1.212921e+07,3.478803e+06,2016-05-05 20:46:10.000,2016-05-05,2016-05-05 20:46:10.000
763,763,4818,900 BLOCK OF E CHARLOTTE STREET,unnamed (2/25/2016),Flooded street,1.213223e+07,3.476292e+06,2016-05-05 20:51:34.000,2016-05-05,2016-05-05 20:51:34.000
764,764,4819,LLEWELLYN AVENUE & W VIRGINIA BEACH BOULEVARD,unnamed (2/25/2016),Flooded street,1.212906e+07,3.479121e+06,2016-05-05 20:52:17.000,2016-05-05,2016-05-05 20:52:17.000
765,765,4820,ORLEANS STREET & LAFAYETTE AVENUE,unnamed (2/25/2016),Flooded street,1.213787e+07,3.513103e+06,2016-05-06 20:51:41.000,2016-05-06,2016-05-06 20:51:41.000
766,766,4821,GRANBY STREET & LLEWELLYN AVENUE,unnamed (2/25/2016),Flooded street,1.213187e+07,3.490070e+06,2016-05-06 20:52:52.000,2016-05-06,2016-05-06 20:52:52.000
767,767,4823,DUKE STREET & W OLNEY ROAD,unnamed (2/25/2016),Flooded street,1.212885e+07,3.478992e+06,2016-05-31 08:45:33.000,2016-05-31,2016-05-31 08:45:33.000


So to look at this more closely, I will calculate the difference in days between the "event" column date and the dates in the "\_date" column.

When I tried to calculate the time between the 'event_date' and the 'dates' to see how far off these were I found that two events had the same 'event_date'. So I think it's appropriate to drop the 'unnamed' one based on the fact that the dates in the "\_date" column are further from the "event_date".

In [15]:
event_df.sort_index(inplace=True)
event_df.loc['2016-07-30']

dates  \
event_date event_name                                                                   
2016-07-30 Thunderstorm-2016-07-30                    [2016-07-30T00:00:00.000000000]   
           unnamed-2016-07-30       [2016-08-02T00:00:00.000000000, 2016-08-31T00:...   

                                    num_flooded  num_dates  
event_date event_name                                       
2016-07-30 Thunderstorm-2016-07-30            4          1  
           unnamed-2016-07-30                 9          2

In [16]:
i = event_df.loc['2016-07-30', 'unnamed-2016-07-30'].name
event_df.drop(i, inplace=True)
i = event_df.loc['2014-09-13', "NAPSG-2014-09-13"].name
event_df.drop(i, inplace=True)

In [17]:
event_df.reset_index(inplace=True)
event_df.set_index('event_date', inplace=True)
event_df

,event_name,dates,num_flooded,num_dates
event_date,,,,
2010-09-30,Nicole-2010-09-30,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...",101,3
2010-12-16,Snow-2010-12-16,[2010-12-16T00:00:00.000000000],2,1
2011-08-27,Irene-2011-08-27,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110,4
2012-10-28,Sandy-2012-10-28,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...",105,3
2013-10-09,Heavy Rain-2013-10-09,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...",36,3
2014-05-16,Heavy Rain-2014-05-16,[2014-05-16T00:00:00.000000000],35,1
2014-06-09,unnamed-2014-06-09,[2014-06-09T00:00:00.000000000],1,1
2014-06-19,Thunderstorms-2014-06-19,[2014-06-20T00:00:00.000000000],5,1
2014-07-03,Arthur-2014-07-03,[2014-07-04T00:00:00.000000000],8,1


In [18]:
days_away = []
max_days = []
for d in event_df.index:
    try:
        ar = event_df.loc[d, 'dates'] - np.datetime64(d)
        ar = ar.astype('timedelta64[D]')
        days = ar / np.timedelta64(1, 'D')
        days_away.append(days)
        max_days.append(days.max())
    except ValueError:
        print d
event_df['days_away_from_event'] = days_away
event_df['max_days_away'] = max_days
print event_df.shape
event_df.head()

(45, 6)


,event_name,dates,num_flooded,num_dates,days_away_from_event,max_days_away
event_date,,,,,,
2010-09-30,Nicole-2010-09-30,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...",101,3,"[0.0, 1.0, 4.0]",4.0
2010-12-16,Snow-2010-12-16,[2010-12-16T00:00:00.000000000],2,1,[0.0],0.0
2011-08-27,Irene-2011-08-27,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110,4,"[-3.0, 0.0, 1.0, 2.0]",2.0
2012-10-28,Sandy-2012-10-28,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...",105,3,"[-1.0, 0.0, 1.0]",1.0
2013-10-09,Heavy Rain-2013-10-09,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...",36,3,"[-1.0, 0.0, 1.0]",1.0


I don't trust the events that have higher days away so I will disregard any event with a "max_days_away" greater than 10. Five events fall under this category.

In [19]:
# event_df = event_df[event_df['max_days_away']<10]
print event_df.shape
event_df

(45, 6)


,event_name,dates,num_flooded,num_dates,days_away_from_event,max_days_away
event_date,,,,,,
2010-09-30,Nicole-2010-09-30,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...",101,3,"[0.0, 1.0, 4.0]",4.0
2010-12-16,Snow-2010-12-16,[2010-12-16T00:00:00.000000000],2,1,[0.0],0.0
2011-08-27,Irene-2011-08-27,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110,4,"[-3.0, 0.0, 1.0, 2.0]",2.0
2012-10-28,Sandy-2012-10-28,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...",105,3,"[-1.0, 0.0, 1.0]",1.0
2013-10-09,Heavy Rain-2013-10-09,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...",36,3,"[-1.0, 0.0, 1.0]",1.0
2014-05-16,Heavy Rain-2014-05-16,[2014-05-16T00:00:00.000000000],35,1,[0.0],0.0
2014-06-09,unnamed-2014-06-09,[2014-06-09T00:00:00.000000000],1,1,[0.0],0.0
2014-06-19,Thunderstorms-2014-06-19,[2014-06-20T00:00:00.000000000],5,1,[1.0],1.0
2014-07-03,Arthur-2014-07-03,[2014-07-04T00:00:00.000000000],8,1,[1.0],1.0


In [20]:
feature_df = get_db_table_as_df('nor_daily_observations', dbfilename=db_filename)
feature_df = pd.read_csv('nor_daily_observations.csv')
feature_df['Datetime'] = pd.to_datetime(feature_df['Datetime'])
feature_df.set_index('Datetime', inplace = True)
feature_df.head()

,WDF2-19,WDF2-20,WSF2-19,WSF2-20,AWDR-19,AWND-19,AWND-20,WGF6-18,AWND-18,AWDR-18,...,r3d-1,r3d-2,r3d-7,r3d-11,r3d-12,r3d-13,r3d-14,r3d-15,r3d-16,r3d-21
Datetime,,,,,,,,,,,,,,,,,,,,,
2010-09-15,140.0,110.0,13.0,15.0,119.0,5.1,5.6,NaN,NaN,NaN,...,NaN,NaN,2.200000e-01,2.400000e-01,2.600000e-01,2.200000e-01,3.100000e-01,2.400000e-01,3.600000e-01,2.400000e-01
2010-09-16,220.0,200.0,25.9,19.9,210.0,15.2,9.6,NaN,NaN,NaN,...,NaN,NaN,1.000000e-02,-3.053113e-16,-2.220446e-16,-3.608225e-16,3.330669e-16,-1.110223e-15,-1.554312e-15,-5.551115e-17
2010-09-17,220.0,200.0,21.0,16.1,41.0,11.0,9.4,NaN,NaN,NaN,...,NaN,NaN,-1.351350e-15,-3.053113e-16,-2.220446e-16,-3.608225e-16,3.330669e-16,-1.110223e-15,-1.554312e-15,-5.551115e-17
2010-09-18,30.0,30.0,16.1,18.1,37.0,8.5,9.4,NaN,NaN,NaN,...,NaN,NaN,-1.351350e-15,-3.053113e-16,-2.220446e-16,-3.608225e-16,3.330669e-16,-1.110223e-15,-1.554312e-15,-5.551115e-17
2010-09-19,40.0,10.0,10.1,12.1,58.0,4.3,4.7,NaN,NaN,NaN,...,NaN,NaN,-1.351350e-15,-3.053113e-16,-2.220446e-16,-3.608225e-16,3.330669e-16,-1.110223e-15,-1.554312e-15,-5.551115e-17


### Combine env. data with event data

In [21]:
def add_event_data(evnt_data, evnt_df, col_name, func, idx):
    res = func(evnt_data[col_name])
    evnt_df.loc[idx, col_name] = res
    return evnt_df

Now for each event we get an aggregate of the different variables for the given dates

In [22]:
event_df = pd.concat([event_df, pd.DataFrame(columns=feature_df.columns)])

for ind in event_df.index:
    # get the dates of the event and include the date in the "event" column
    ds = event_df.loc[ind, 'dates']
    ind = np.datetime64(ind)
    ds = np.append(ds, ind) if not ind in ds else ds
    
    event_data = feature_df.loc[ds]
 
    # combining data on event scale
    # get max over the event for these features
    max_cols = ['rhrmx', 'r15mx', 'wind_vel_daily_avg', 'wind_vel_hourly_max_avg', 'ht', 'hht', 'lt', 'llt']
    
    # get mean over the event for these features
    mean_cols = ['W', 'td', 'gw', 'AWDR', 'AWND']
    
    # get sum over the event for these features
    sum_cols = ['rd']
    
    for feat in feature_df.columns:
        if any(feat.startswith(col) for col in max_cols):
            event_df = add_event_data(event_data, event_df, feat, np.max, ind)
        elif any(feat.startswith(col) for col in mean_cols):
            event_df = add_event_data(event_data, event_df, feat, np.mean, ind)
        elif any(feat.startswith(col) for col in sum_cols):
            event_df = add_event_data(event_data, event_df, feat, np.sum, ind)
        elif feat.startswith('r3d'):
            event_df.loc[ind, feat] = event_data.loc[ind, feat]
        elif re.search(re.compile(r'r\w{2}-\d+_td-\d+'), feat):
            feat_spl = feat.split('-')
            var = '{}mx-{}'.format(feat_spl[0], feat_spl[1].split('_')[0])
            max_ind = event_data[var].idxmax()
            if isinstance(max_ind, float):
                if math.isnan(max_ind):
                    event_df.loc[ind, feat] = np.nan
            else:
                val = event_data.loc[max_ind, feat]
                event_df.loc[ind, feat] = event_data.loc[max_ind, feat]
        
event_df.head()

,AWDR-18,AWDR-19,AWDR-3,AWND-18,AWND-19,AWND-20,AWND-3,WDF2-19,WDF2-20,WGF6-18,...,rhrmx-12,rhrmx-13,rhrmx-14,rhrmx-15,rhrmx-16,rhrmx-2,rhrmx-21,rhrmx-7,td_av-17,td_av-18
2010-09-30,NaN,291.667,NaN,NaN,11.6,12.6,NaN,83.3333,160,NaN,...,1.59,1.4,1.17,0.9,1.31,NaN,1.49,1.43,1.38399,1.52196
2010-12-16,NaN,213,NaN,NaN,2.9,2.5,NaN,210,200,NaN,...,0.07,0.05,0.12,0.06,0.12,NaN,0.03,0.11,-0.208504,-0.195775
2011-08-27,133.611,133,NaN,8.5641,13.975,14.725,NaN,130,187.5,13.9539,...,1.22,1.12,0.93,1.02,1.3,NaN,1.1,1.03,1.04581,1.14569
2012-10-28,110.73,123,107.629,11.924,23.8,24,16.1683,20,16.6667,20.0615,...,0.58,0.4,0.38,0.52,0.43,NaN,0.48,0.23,2.70266,2.79603
2013-10-09,41.5125,25.6667,52.9563,8.7694,20.8,21.5667,12.8631,36.6667,26.6667,15.3151,...,0.45,0.47,3.07,0.56,0.4,NaN,0.46,0.5,2.2125,2.27723


In [23]:
event_df.shape

(45, 119)

In [24]:
event_df.head()

,AWDR-18,AWDR-19,AWDR-3,AWND-18,AWND-19,AWND-20,AWND-3,WDF2-19,WDF2-20,WGF6-18,...,rhrmx-12,rhrmx-13,rhrmx-14,rhrmx-15,rhrmx-16,rhrmx-2,rhrmx-21,rhrmx-7,td_av-17,td_av-18
2010-09-30,NaN,291.667,NaN,NaN,11.6,12.6,NaN,83.3333,160,NaN,...,1.59,1.4,1.17,0.9,1.31,NaN,1.49,1.43,1.38399,1.52196
2010-12-16,NaN,213,NaN,NaN,2.9,2.5,NaN,210,200,NaN,...,0.07,0.05,0.12,0.06,0.12,NaN,0.03,0.11,-0.208504,-0.195775
2011-08-27,133.611,133,NaN,8.5641,13.975,14.725,NaN,130,187.5,13.9539,...,1.22,1.12,0.93,1.02,1.3,NaN,1.1,1.03,1.04581,1.14569
2012-10-28,110.73,123,107.629,11.924,23.8,24,16.1683,20,16.6667,20.0615,...,0.58,0.4,0.38,0.52,0.43,NaN,0.48,0.23,2.70266,2.79603
2013-10-09,41.5125,25.6667,52.9563,8.7694,20.8,21.5667,12.8631,36.6667,26.6667,15.3151,...,0.45,0.47,3.07,0.56,0.4,NaN,0.46,0.5,2.2125,2.27723


In [25]:
cols = event_df.columns.tolist()
lft_cols = ['event_name', 'dates', 'num_flooded', 'days_away_from_event', 'max_days_away', 'num_dates']
lft_cols.reverse()
for c in lft_cols:
    cols.insert(0, cols.pop(cols.index(c)))
event_df = event_df.loc[:, cols]
event_df_for_storage = event_df.reset_index()
event_df_for_storage['dates'] = event_df_for_storage['dates'].apply(str)
event_df_for_storage['days_away_from_event'] = event_df_for_storage['days_away_from_event'].apply(str)
event_df_for_storage.rename(columns={'index':'event_date'}, inplace=True)
event_df_for_storage.head()

,event_date,event_name,dates,num_flooded,days_away_from_event,max_days_away,num_dates,AWDR-18,AWDR-19,AWDR-3,...,rhrmx-12,rhrmx-13,rhrmx-14,rhrmx-15,rhrmx-16,rhrmx-2,rhrmx-21,rhrmx-7,td_av-17,td_av-18
0,2010-09-30,Nicole-2010-09-30,['2010-09-30T00:00:00.000000000' '2010-10-01T0...,101.0,[ 0. 1. 4.],4.0,3.0,NaN,291.667,NaN,...,1.59,1.4,1.17,0.9,1.31,NaN,1.49,1.43,1.38399,1.52196
1,2010-12-16,Snow-2010-12-16,['2010-12-16T00:00:00.000000000'],2.0,[ 0.],0.0,1.0,NaN,213,NaN,...,0.07,0.05,0.12,0.06,0.12,NaN,0.03,0.11,-0.208504,-0.195775
2,2011-08-27,Irene-2011-08-27,['2011-08-24T00:00:00.000000000' '2011-08-27T0...,110.0,[-3. 0. 1. 2.],2.0,4.0,133.611,133,NaN,...,1.22,1.12,0.93,1.02,1.3,NaN,1.1,1.03,1.04581,1.14569
3,2012-10-28,Sandy-2012-10-28,['2012-10-27T00:00:00.000000000' '2012-10-28T0...,105.0,[-1. 0. 1.],1.0,3.0,110.73,123,107.629,...,0.58,0.4,0.38,0.52,0.43,NaN,0.48,0.23,2.70266,2.79603
4,2013-10-09,Heavy Rain-2013-10-09,['2013-10-08T00:00:00.000000000' '2013-10-09T0...,36.0,[-1. 0. 1.],1.0,3.0,41.5125,25.6667,52.9563,...,0.45,0.47,3.07,0.56,0.4,NaN,0.46,0.5,2.2125,2.27723


In [26]:
event_df_for_storage.to_csv('{}event_data.csv'.format(data_dir), index=False)
event_df_for_storage.to_sql(name='event_data', con=con, if_exists='replace', index=False)

### Combining with the non-flooding event data
First we have to combine all the dates in the "dates" column of the event_df into one array so we can filter those out of the overall dataset.

In [27]:
flooded_dates = [np.datetime64(i) for i in event_df.index]
flooded_dates = np.array(flooded_dates)
fl_event_dates = np.concatenate(event_df['dates'].tolist())
all_fl_dates = np.concatenate([fl_event_dates, flooded_dates])

In [28]:
non_flooded_records = feature_df[feature_df.index.isin(all_fl_dates) != True]
non_flooded_records['num_flooded'] = 0
non_flooded_records['flooded'] = False
non_flooded_records['event_name'] = np.nan
non_flooded_records['event_date'] = non_flooded_records.index
non_flooded_records.reset_index(drop=True, inplace=True)
non_flooded_records.head()

,WDF2-19,WDF2-20,WSF2-19,WSF2-20,AWDR-19,AWND-19,AWND-20,WGF6-18,AWND-18,AWDR-18,...,r3d-12,r3d-13,r3d-14,r3d-15,r3d-16,r3d-21,num_flooded,flooded,event_name,event_date
0,140.0,110.0,13.0,15.0,119.0,5.1,5.6,NaN,NaN,NaN,...,2.600000e-01,2.200000e-01,3.100000e-01,2.400000e-01,3.600000e-01,2.400000e-01,0,False,NaN,2010-09-15
1,220.0,200.0,25.9,19.9,210.0,15.2,9.6,NaN,NaN,NaN,...,-2.220446e-16,-3.608225e-16,3.330669e-16,-1.110223e-15,-1.554312e-15,-5.551115e-17,0,False,NaN,2010-09-16
2,220.0,200.0,21.0,16.1,41.0,11.0,9.4,NaN,NaN,NaN,...,-2.220446e-16,-3.608225e-16,3.330669e-16,-1.110223e-15,-1.554312e-15,-5.551115e-17,0,False,NaN,2010-09-17
3,30.0,30.0,16.1,18.1,37.0,8.5,9.4,NaN,NaN,NaN,...,-2.220446e-16,-3.608225e-16,3.330669e-16,-1.110223e-15,-1.554312e-15,-5.551115e-17,0,False,NaN,2010-09-18
4,40.0,10.0,10.1,12.1,58.0,4.3,4.7,NaN,NaN,NaN,...,-2.220446e-16,-3.608225e-16,3.330669e-16,-1.110223e-15,-1.554312e-15,-5.551115e-17,0,False,NaN,2010-09-19


Combine with flooded events

In [29]:
event_df.reset_index(inplace=True)
flooded_records = event_df
flooded_records['event_date'] = event_df['index']
flooded_records['flooded'] = True
flooded_records.head()

,index,event_name,dates,num_flooded,days_away_from_event,max_days_away,num_dates,AWDR-18,AWDR-19,AWDR-3,...,rhrmx-14,rhrmx-15,rhrmx-16,rhrmx-2,rhrmx-21,rhrmx-7,td_av-17,td_av-18,event_date,flooded
0,2010-09-30,Nicole-2010-09-30,"[2010-09-30T00:00:00.000000000, 2010-10-01T00:...",101.0,"[0.0, 1.0, 4.0]",4.0,3.0,NaN,291.667,NaN,...,1.17,0.9,1.31,NaN,1.49,1.43,1.38399,1.52196,2010-09-30,True
1,2010-12-16,Snow-2010-12-16,[2010-12-16T00:00:00.000000000],2.0,[0.0],0.0,1.0,NaN,213,NaN,...,0.12,0.06,0.12,NaN,0.03,0.11,-0.208504,-0.195775,2010-12-16,True
2,2011-08-27,Irene-2011-08-27,"[2011-08-24T00:00:00.000000000, 2011-08-27T00:...",110.0,"[-3.0, 0.0, 1.0, 2.0]",2.0,4.0,133.611,133,NaN,...,0.93,1.02,1.3,NaN,1.1,1.03,1.04581,1.14569,2011-08-27,True
3,2012-10-28,Sandy-2012-10-28,"[2012-10-27T00:00:00.000000000, 2012-10-28T00:...",105.0,"[-1.0, 0.0, 1.0]",1.0,3.0,110.73,123,107.629,...,0.38,0.52,0.43,NaN,0.48,0.23,2.70266,2.79603,2012-10-28,True
4,2013-10-09,Heavy Rain-2013-10-09,"[2013-10-08T00:00:00.000000000, 2013-10-09T00:...",36.0,"[-1.0, 0.0, 1.0]",1.0,3.0,41.5125,25.6667,52.9563,...,3.07,0.56,0.4,NaN,0.46,0.5,2.2125,2.27723,2013-10-09,True


In [30]:
reformat = pd.concat([flooded_records, non_flooded_records], join='inner')
reformat.reset_index(inplace=True, drop=True)
reformat.head()

,event_name,num_flooded,AWDR-18,AWDR-19,AWDR-3,AWND-18,AWND-19,AWND-20,AWND-3,WDF2-19,...,rhrmx-14,rhrmx-15,rhrmx-16,rhrmx-2,rhrmx-21,rhrmx-7,td_av-17,td_av-18,event_date,flooded
0,Nicole-2010-09-30,101.0,NaN,291.667,NaN,NaN,11.6,12.6,NaN,83.3333,...,1.17,0.9,1.31,NaN,1.49,1.43,1.38399,1.52196,2010-09-30,True
1,Snow-2010-12-16,2.0,NaN,213,NaN,NaN,2.9,2.5,NaN,210,...,0.12,0.06,0.12,NaN,0.03,0.11,-0.208504,-0.195775,2010-12-16,True
2,Irene-2011-08-27,110.0,133.611,133,NaN,8.5641,13.975,14.725,NaN,130,...,0.93,1.02,1.3,NaN,1.1,1.03,1.04581,1.14569,2011-08-27,True
3,Sandy-2012-10-28,105.0,110.73,123,107.629,11.924,23.8,24,16.1683,20,...,0.38,0.52,0.43,NaN,0.48,0.23,2.70266,2.79603,2012-10-28,True
4,Heavy Rain-2013-10-09,36.0,41.5125,25.6667,52.9563,8.7694,20.8,21.5667,12.8631,36.6667,...,3.07,0.56,0.4,NaN,0.46,0.5,2.2125,2.27723,2013-10-09,True


In [31]:
reformat.to_sql(name="for_model", con=con, index=False, if_exists='replace')

## Make average table

In [32]:
cols = pd.Series(feature_df.columns)
cols_splt = cols.str.split('-', expand=True)
# do this to make sure the tide when the hourly and 15-min max rains have unique col names
for a in cols_splt.iterrows():
    if a[1].str.contains('\d_td').sum() == 1:
        cols_splt.loc[a[0], 0] += "_td"
col_vars = cols_splt[0].unique()
col_vars

array(['WDF2', 'WSF2', 'AWDR', 'AWND', 'WGF6', 'ht', 'hht', 'lt', 'llt',
       'r15_td', 'rhr_td', 'td_av', 'gw_av', 'r15mx', 'rhrmx', 'rd', 'r3d'], dtype=object)

In [33]:
avdf = pd.DataFrame()
for v in col_vars:
    if v not in ['r15_td', 'rhr_td']:
        avdf[v] = reformat[[a for a in feature_df.columns if a.startswith(v)]].mean(axis=1)
    else:
        avdf[v] = reformat[cols[cols.str.contains(r'{}-\d+_td-\d+'.format(v.split('_')[0]))]].mean(axis=1)

In [34]:
avdf = pd.concat([reformat[['event_date', 'event_name', 'num_flooded']], avdf], axis=1)

In [35]:
avdf.head()

,event_date,event_name,num_flooded,WDF2,WSF2,AWDR,AWND,WGF6,ht,hht,lt,llt,r15_td,rhr_td,td_av,gw_av,r15mx,rhrmx,rd,r3d
0,2010-09-30,Nicole-2010-09-30,101.0,121.666667,25.966667,291.666667,12.100000,NaN,3.2105,3.6175,1.1860,0.5330,1.156938,0.926854,1.452974,4.340699,0.605,1.32250,7.732500,3.518
1,2010-12-16,Snow-2010-12-16,2.0,205.000000,15.000000,213.000000,2.700000,NaN,0.6610,0.8610,-1.1300,-1.2235,-0.039190,0.132667,-0.202140,2.208326,0.030,0.07000,0.227500,0.045
2,2011-08-27,Irene-2011-08-27,110.0,158.750000,27.125000,133.305686,12.421368,13.953947,2.2375,4.9680,2.6985,-0.6925,3.204875,3.158667,1.095753,2.334049,0.350,1.10125,6.294167,0.308
3,2012-10-28,Sandy-2012-10-28,105.0,18.333333,38.316667,113.786208,18.973074,20.061461,5.2280,5.5760,3.2840,1.5810,4.753823,4.328802,2.749344,2.998493,0.140,0.42625,4.805833,0.341
4,2013-10-09,Heavy Rain-2013-10-09,36.0,31.666667,32.550000,40.045150,15.999780,15.315069,3.4925,4.2125,2.0830,1.0845,3.835031,3.914865,2.244865,2.141837,0.375,0.80000,3.671667,0.924


In [36]:
avdf['ht'] = np.where(avdf['ht'].isnull(), avdf['hht'], avdf['ht'])
avdf['hht'] = np.where(avdf['hht'].isnull(), avdf['ht'], avdf['hht'])
avdf['lt'] = np.where(avdf['lt'].isnull(), avdf['llt'], avdf['lt'])
avdf['llt'] = np.where(avdf['llt'].isnull(), avdf['lt'], avdf['llt'])
avdf['WGF6'] = np.where(avdf['WGF6'].isnull(), avdf['AWND'], avdf['WGF6'])

In [37]:
avdf.to_sql(name='for_model_avgs', con=con, index=False, if_exists='replace')
avdf.to_csv('for_model_avgs.csv')

In [38]:
avdf

,event_date,event_name,num_flooded,WDF2,WSF2,AWDR,AWND,WGF6,ht,hht,lt,llt,r15_td,rhr_td,td_av,gw_av,r15mx,rhrmx,rd,r3d
0,2010-09-30,Nicole-2010-09-30,101.0,121.666667,25.966667,291.666667,12.100000,12.100000,3.21050,3.61750,1.18600,0.53300,1.156938,0.926854,1.452974,4.340699,0.605000,1.322500e+00,7.732500,3.518000e+00
1,2010-12-16,Snow-2010-12-16,2.0,205.000000,15.000000,213.000000,2.700000,2.700000,0.66100,0.86100,-1.13000,-1.22350,-0.039190,0.132667,-0.202140,2.208326,0.030000,7.000000e-02,0.227500,4.500000e-02
2,2011-08-27,Irene-2011-08-27,110.0,158.750000,27.125000,133.305686,12.421368,13.953947,2.23750,4.96800,2.69850,-0.69250,3.204875,3.158667,1.095753,2.334049,0.350000,1.101250e+00,6.294167,3.080000e-01
3,2012-10-28,Sandy-2012-10-28,105.0,18.333333,38.316667,113.786208,18.973074,20.061461,5.22800,5.57600,3.28400,1.58100,4.753823,4.328802,2.749344,2.998493,0.140000,4.262500e-01,4.805833,3.410000e-01
4,2013-10-09,Heavy Rain-2013-10-09,36.0,31.666667,32.550000,40.045150,15.999780,15.315069,3.49250,4.21250,2.08300,1.08450,3.835031,3.914865,2.244865,2.141837,0.375000,8.000000e-01,3.671667,9.240000e-01
5,2014-05-16,Heavy Rain-2014-05-16,35.0,200.000000,22.450000,217.404042,6.933247,9.721000,1.84200,2.63900,-0.98425,-0.98425,0.614093,0.818426,0.560302,3.620773,0.498889,1.413333e+00,2.923333,1.272727e-02
6,2014-06-09,unnamed-2014-06-09,1.0,170.000000,16.000000,197.291344,7.485292,8.440377,1.04650,1.80950,-0.88900,-1.01400,0.772417,0.805500,0.306098,2.362913,0.015556,1.888889e-02,0.016667,2.727273e-03
7,2014-06-19,Thunderstorms-2014-06-19,5.0,152.500000,20.800000,172.140479,6.515842,7.648271,1.85100,2.11200,-0.74300,-0.76450,1.340093,1.100796,0.581874,2.432714,0.556667,8.611111e-01,0.747500,2.727273e-03
8,2014-07-03,Arthur-2014-07-03,8.0,260.000000,25.775000,222.405868,11.009788,13.990583,1.83550,2.14250,0.66950,0.01800,1.343537,1.352315,0.668728,2.162596,0.223333,3.833333e-01,1.039167,1.363636e-02
9,2014-07-09,Thunderstorms-2014-07-09,2.0,260.000000,34.000000,224.827356,9.101778,10.333042,0.38750,1.35500,-1.76125,-1.76125,0.903926,0.982981,-0.288448,0.710830,0.463333,9.066667e-01,1.367500,2.727273e-03
